## Import

In [1]:
import sys; sys.path.insert(0, '..')
import os
import time
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tabulate import tabulate
from dataclasses import dataclass
from keras.models import Sequential, load_model
from keras.layers import Conv1D, ZeroPadding1D, AveragePooling1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam
from keras.losses import CategoricalCrossentropy
from keras import activations
from sklearn.metrics import confusion_matrix
from utils.df_utils import df_wrapper
from utils.declarations import training_files, testing_files, POSE_MAP

## Define data

In [2]:
AMOUNT_OF_SENSORS = 3
VALIDATION_TESTSET = "004"

## Load dataframes

In [3]:
pd.set_option('io.hdf.default.format', 'table')

x_train_arr = list()
for key in training_files:
    x_train_arr.append(pd.read_hdf(f"../../data/preprocessed_data/train_arr_{AMOUNT_OF_SENSORS}.h5", f"df{key}"))

x_test_dict = dict()
y_test_dict = dict()
for key in testing_files:
    x_test_dict[key] = pd.read_hdf(f"../../data/preprocessed_data/test_dict_{AMOUNT_OF_SENSORS}.h5", f"df{key}")
    y_test = x_test_dict[key]["Pose"]
    y_test.index = [i for i in range(len(y_test))]
    y_test_dict[key] = y_test

x_train = pd.concat([df.drop([' TimeStamp (s)', 'Pose'],axis=1) for df in x_train_arr])
y_train = pd.concat([df['Pose'] for df in x_train_arr])
x_test = x_test_dict[VALIDATION_TESTSET].drop([' TimeStamp (s)', 'Pose'], axis=1) 
y_test = y_test_dict[VALIDATION_TESTSET]


## Adjust dataframes

In [4]:
# Create numpy arrays
x_train_numpy = x_train.values
x_test_numpy = x_test.values
y_train_numpy = y_train.values
y_test_numpy = y_test.values

# Hot encode categories into numbers
y_train = to_categorical(y_train_numpy,9)
y_test = to_categorical(y_test_numpy,9)

# Change shape for CNN
x_train_numpy = x_train_numpy.reshape(x_train_numpy.shape[0],x_train_numpy.shape[1], 1)
x_test_numpy = x_test_numpy.reshape(x_test_numpy.shape[0],x_test_numpy.shape[1], 1)

## Hyperparams

In [5]:
EPOCH = 60
BATCH_SIZE = 192 # 160
LR = 0.000005 #0.000005
OPTIM = Adam(learning_rate=LR)
#OPTIM = RMSprop(learning_rate=LR)

# Shapes
INPUT_SHAPE = (x_train_numpy.shape[1], 1)
OUTPUT_SHAPE = 9

## Model

In [ ]:
ids = [2]
AMOUNT_OF_MODELS = 1
PRINT_HISTORY_PLOT = True
SAVE_MODEL = True

for run_i in ids:
    print(f"""#####################
#####   Run {run_i}   #####
#####################""")
    model = Sequential()
    model.add(BatchNormalization(input_shape=INPUT_SHAPE))
    model.add(Conv1D(128, kernel_size=4, padding="same", activation=activations.relu))
    model.add(Conv1D(64, kernel_size=4, padding="same", activation=activations.relu))
    model.add(Conv1D(32, kernel_size=4, padding="same", activation=activations.relu))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=(2))) #2
    model.add(Flatten())
    model.add(Dense(64, activation=activations.relu))
    model.add(Dropout(0.2))
    model.add(Dense(OUTPUT_SHAPE, activation=activations.softmax))

    model.summary()
    model.compile(optimizer=OPTIM, loss=CategoricalCrossentropy(), metrics=['accuracy'])
    model.fit(x=x_train_numpy, y=y_train, batch_size=BATCH_SIZE, validation_data=(x_test_numpy, y_test), shuffle=True, epochs=EPOCH)

    if PRINT_HISTORY_PLOT:
        history = pd.DataFrame(model.history.history)
        history.plot()

    print("\nClassification accuracy:")
    classification_dict = dict()
    accuracy_list = list() 

    for key in x_test_dict:
        x_numpy = x_test_dict[key].drop([' TimeStamp (s)', 'Pose'], axis=1).values
        x_numpy = x_numpy.reshape(x_numpy.shape[0],x_numpy.shape[1], 1)
        classify = model.predict(x_numpy)
        classifications = [i.argmax() for i in classify]
        annotated_positions = y_test_dict[key].to_numpy()
        correct_classifications = (classifications == annotated_positions).sum()
        accuracy_list.append(round(correct_classifications/len(classifications)*100,2))
        print(f"{key}: {accuracy_list[-1]}%")
        classification_dict[key] = classifications

    print(f"Average accuracy: {round(sum(accuracy_list)/len(accuracy_list),2)}%\n")

    if SAVE_MODEL:
        path = f"../models/CNN_model_{AMOUNT_OF_SENSORS}_{run_i}.h5"
        model.save(path)

#####################
#####   Run 2   #####
#####################
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 39, 1)             4         
_________________________________________________________________
conv1d (Conv1D)              (None, 39, 128)           640       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 64)            32832     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 39, 32)            8224      
_________________________________________________________________
dropout (Dropout)            (None, 39, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 32)            0         
________________________________________________________

2433/2433 [==============================] - 159s 65ms/step - loss: 0.1543 - accuracy: 0.9553 - val_loss: 0.1742 - val_accuracy: 0.9079
Epoch 29/60
 199/2433 [=>............................] - ETA: 2:31 - loss: 0.1809 - accuracy: 0.9456

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=59d486bc-e14d-4632-9064-12272fc72d11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>